In [1]:
# !pip install ImageHash

In [19]:
from pymongo import MongoClient
import os
from pathlib import Path
from PIL import Image
import imagehash
from tqdm.notebook import tqdm
import json

from lib.parallel import parallel
from lib.sort_things import simple_sort_images
from lib.image_dedup import make_hashes

In [3]:
from lib.fix_nested_tqdm import fix_nested_tqdm

fix_nested_tqdm()

In [4]:
mongo = MongoClient('172.17.0.1', 27017)
db = mongo['bad-vis']
imagefiles = db['imagefiles']

In [5]:
imagefiles.drop()

In [6]:
images_dir = Path('./images')

In [7]:
class ImageFile ():
    _attrs = [
        'id',
        'post_id',
        'image_id',
        'short_image_id',
        'album',
        'index_in_album',
        'image_type',
        'file_path',
        'filename',
        'basename',
        'ext',
        'animated',
        'source_platform',
        'source',
        'size',
        'width',
        'height',
        'pixels',
        'ahash',
        'phash',
        'pshash',
        'dhash',
        'whash'
    ]

    def __init__ (self, file_path):
        self.file_path = file_path
        self.filename = os.path.basename(file_path)
        self.basename, self.ext = os.path.splitext(self.filename)
        self._im = Image.open(file_path)
        self.width, self.height = self._im.size
        self.pixels = self.width * self.height

        self._file_path_tokens = self.file_path.split('/')

    def digest (self):
        return {a:getattr(self, a) for a in ImageFile._attrs}

    @property
    def size (self):
        return os.path.getsize(self.file_path)

    @property
    def animated (self):
        return self._im.is_animated if self._im.format == 'GIF' or self._im.format == 'WEBP' else False

    @property
    def image_type (self):
        return self._file_path_tokens[1]

    @property
    def source_platform (self):
        return self._file_path_tokens[2]

    @property
    def source (self):
        return self._file_path_tokens[3]

    @property
    def album (self):
        if '-' in self.basename:
            if len(self._file_path_tokens) > 5:
                return self._file_path_tokens[4]
            else:
                return self.basename.split('-')[0]
        else:
            return ''

    @property
    def index_in_album (self):
        if '-' not in self.basename:
            return 0
        else:
            p1, p2 = self.basename.split('-')
            return int(p1) if len(p1) < len(p2) else int(p2)

    @property
    def id (self):
        return self.album if self.album else self.basename

    @property
    def post_id (self):
        return f"{self.source_platform}/{self.source}/{self.id}"

    @property
    def image_id (self):
        return f"{self.post_id}_{self.index_in_album}"

    @property
    def short_image_id (self):
        return f"{self.id}_{self.index_in_album}"

    @property
    def ahash (self):
        if not hasattr(self, '_ahash'):
            self._ahash = imagehash.average_hash(self._im)
        return str(self._ahash)

    @property
    def phash (self):
        if not hasattr(self, '_phash'):
            self._phash = imagehash.phash(self._im)
        return str(self._phash)

    @property
    def pshash (self):
        if not hasattr(self, '_pshash'):
            self._pshash = imagehash.phash_simple(self._im)
        return str(self._pshash)

    @property
    def dhash (self):
        if not hasattr(self, '_dhash'):
            self._dhash = imagehash.dhash(self._im)
        return str(self._dhash)

    @property
    def whash (self):
        if not hasattr(self, '_whash'):
            self._whash = imagehash.whash(self._im)
        return str(self._whash)

In [8]:
def digest_file (name, root=''):
    basename, ext = os.path.splitext(name)
    imageFile = ImageFile(os.path.join(root, name))
    if not imagefiles.find_one({'file_path': imageFile.file_path}, {'_id': 1}):
        imagefiles.insert_one(imageFile.digest())

with tqdm(os.walk(images_dir)) as t:
    for root, dirs, files in t:
        t.set_postfix(root=root)
        parallel(digest_file, files, params_dict={'root': root}, tqdm_postfix=root, leave=None)

/opt/conda/lib/python3.7/site-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


/opt/conda/lib/python3.7/site-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (135364608 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


/opt/conda/lib/python3.7/site-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (100000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


In [9]:
# import warnings
# warnings.filterwarnings("error")
# %%capture cap_out --no-stderr
# for root, dirs, files in os.walk(images_dir):
#     for name in files:
#         try:
#             imageFile = ImageFile(os.path.join(root, name))
#             imageFile.digest()
#         except Exception as inst:
#             print(f"Error digesting image: {os.path.join(root, name)} {inst}")

In [10]:
# with open('error.txt', 'w') as f:
#     f.write(cap_out.stdout)

In [11]:
from IPython.display import Image, display
from ipywidgets import widgets, HBox, Box, Layout

In [12]:
invalid_image_phashes = set(json.load(open('handmade/invalid_image_phashes.json')))

In [13]:
post_ids = {f['post_id'] for f in [f for f in imagefiles.find({'index_in_album': {'$ne': 0}})]}
len(post_ids)

105

In [14]:
processed_post_ids = set()

In [15]:
examined_images = [
    'reddit/dataisugly/2o08rl_0', # manually downloaded
    'reddit/dataisugly/2nwubr_0', # manually downloaded
    'reddit/dataisugly/beivt8_0', # manually downloaded
    'reddit/dataisugly/683b4i_0', # manually downloaded
    'reddit/dataisugly/3zcw30_0', # manually downloaded
    'reddit/dataisugly/1oxrh5_0', # manually downloaded a higher resolution image
    'reddit/dataisugly/3or2g0_0', # manually downloaded
    'reddit/dataisugly/5iobqn_0', # manually downloaded
    'reddit/dataisugly/29fpuo_0', # manually downloaded
    'reddit/dataisugly/5xux1f_0', # manually downloaded
    'reddit/dataisugly/35lrw1_0', # manually downloaded
    'reddit/dataisugly/1bxhv2_0', # manually downloaded a higher resolution image
    'reddit/dataisugly/3peais_0', # manually downloaded
    'reddit/dataisugly/2vdk71_0', # manually downloaded
    'reddit/dataisugly/6b8w73_0', # manually downloaded
    'reddit/dataisugly/2w8pnr_0', # manually downloaded an image with more context
    'reddit/dataisugly/2dt19h_0', # manually downloaded
    'reddit/dataisugly/31tj8a_0', # manually downloaded
    'reddit/dataisugly/30smxr_0', # manually downloaded
    'reddit/dataisugly/30dbx6_0', # manually downloaded
    'reddit/dataisugly/561ytm_0', # manually downloaded
    'reddit/dataisugly/6q4tre_0', # manually downloaded
    'reddit/dataisugly/3icm4g_0', # manually downloaded
    'reddit/dataisugly/6z5v98_0', # manually downloaded
    'reddit/dataisugly/5fucjm_0', # manually downloaded
    'reddit/dataisugly/99bczz_0', # manually downloaded
    'reddit/dataisugly/2662wv_0', # manually downloaded
    'reddit/dataisugly/26otpi_0', # manually downloaded a higher resolution image
    'reddit/dataisugly/68scgb_0', # manually downloaded
    'reddit/dataisugly/et75qp_0', # manually downloaded
    'reddit/dataisugly/4c9zc1_0', # manually downloaded an image with more context
    'reddit/dataisugly/2525a5_0', # manually downloaded more images, but does not matched with the one with more context
    'reddit/dataisugly/2la7zt_0', # thumbnail alt
]

In [21]:
# for i in post_ids:
#     if i in processed_post_ids:
#         continue
#     related_image_files = [f for f in imagefiles.find({'post_id': i})]
#     previews = [r for r in related_image_files if r['index_in_album'] == 0]

#     if len(previews) < 1:
#         print('No preview image', i)
#         continue

#     previews = simple_sort_images(previews)
#     preview = previews[0]

#     album = [r for r in related_image_files if r['index_in_album'] != 0]
#     preview_hashes = make_hashes(preview)
#     album_hashes = [make_hashes(a) for a in album]

#     distance = [hashes_diff(preview_hashes, ah) for ah in album_hashes]
#     min_index = np.argmin(np.asarray(distance))
# #     print(min_index, distance[min_index], preview['image_id'], preview['image_type'])
#     if distance[min_index] >= 4 and preview['image_id'] not in examined_images and preview['phash'] not in invalid_image_phashes:
#         print(min_index, distance[min_index], preview['image_id'], preview['phash'], preview['image_type'], distance)
#         display(Image(filename=preview['file_path']), Image(filename=album[min_index]['file_path']))
#         break
#     for p in previews:
#         new_album = album[min_index]['album']
#         new_index_in_album = album[min_index]['index_in_album']
#         imagefiles.update_one({'file_path': p['file_path']},
#                               {'$set': {
#                                   'album': new_album,
#                                   'index_in_album': new_index_in_album,
#                                   'image_id': p['image_id'].replace('_0', f"_{new_index_in_album}"),
#                                   'short_image_id': p['short_image_id'].replace('_0', f"_{new_index_in_album}")
#                               }})

In [24]:
invalids = []
for h in invalid_image_phashes:
    invalid_images = [f for f in imagefiles.find({'phash': h})]
#     print(f"{h} {[i['image_id'] for i in invalid_images]}")
#     display(HBox([
#             widgets.Image(value=open(i['file_path'], 'rb').read(), width=100, height=100)
#             for i in invalid_images]))
    if len(invalid_images) > 0:
        invalids.append(invalid_images[0])

In [25]:
display(Box([widgets.Image(value=open(i['file_path'], 'rb').read(), width=100, height=100) for i in invalids],
                layout=Layout(display='flex', flex_flow='row wrap')))

Box(children=(Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\…

In [ ]:
# duplicated_image_ids = [c
#                      for c in nx.components.connected_components(nx.Graph(distance <= 1))
#                      if len(c) > 1]
# len(duplicated_image_ids)
# for idxs in duplicated_image_ids:
#     print(f"{[imageDedup[i]['image_id'] for i in idxs]}")
#     if len(idxs) >= 4:
#         display(HBox([
#             widgets.Image(value=open(imageDedup[i]['file_path'], 'rb').read(), width=100, height=100)
#             for i in idxs]))